In [1]:
### Import the required libraries
import numpy as np
import scipy
import matplotlib.pyplot as plt
import cmocean

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter("ignore")

import tensorflow as tf
import tensorflow.keras as keras
from keras import metrics
import innvestigate
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

import os
from os.path import join
import sys

import xarray as xr
import xmitgcm
from xmitgcm import open_mdsdataset
import ecco_v4_py as ecco

import random

# See if GPUs are available
from keras import backend as K
if bool(K._get_available_gpus()):
    print("Running on GPU")
else:
    print("Running on CPU")

# Append to sys.path the absolute path to src/XAIRT
path_list = os.path.abspath('').split('/')
path_src_XAIRT = ''
for link in path_list[:-1]:
    path_src_XAIRT = path_src_XAIRT+link+'/'
sys.path.append(path_src_XAIRT+'/src')

# Now import module XAIRT
from XAIRT import *

### https://stackoverflow.com/questions/36288235/how-to-get-stable-results-with-tensorflow-setting-random-seed ###
### https://keras.io/examples/keras_recipes/reproducibility_recipes/ ###
SEED = 1997
keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()

tf.compat.v1.disable_eager_execution()

2024-12-09 20:25:59.260855: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-12-09 20:26:03.491125: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-12-09 20:26:03.493767: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-12-09 20:26:03.496945: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

Running on GPU


In [2]:
### Sverdrup 
# mainDir_r4  = '/scratch2/pillarh/eccov4r4'
# mainDir_r5  = '/scratch2/pillarh/eccov4r5'
# gridDir  = mainDir_r4 + '/GRID'
# thetaDir = mainDir_r5 + '/V4r5/diags_daily/SST_day_mean'
# thetaDir_ext = mainDir_r5 + '/V4r5_ext_2020_2023_Jun/diags_daily/SST_day_mean'

## LS6
mainDir_r4 = '/work/07665/shrey911/ls6/LRP_eccov4r4_data'
mainDir_r5 = '/work/07665/shrey911/ls6/LRP_eccov4r5_data'
gridDir  = mainDir_r5 + '/GRID'
thetaDir = mainDir_r5 + '/SST_day_mean'
thetaDir_ext = mainDir_r5 + '/SST_day_mean_ext_2020_2023_Jun'

# For Sverdrup
# ds_r4 = xr.open_dataset(f'/scratch2/shreyas/LRP_eccov4r4_data/thetaSurfECCOv4r4.nc')
# For LS6
ds_r4 = xr.open_dataset(mainDir_r4 + '/thetaSurfECCOv4r4.nc')

# SSH has to be kept because someone used the SSH metadata for SST, 
# It's not a bug in this code but a hack to handle an existing bug.
temp = xmitgcm.open_mdsdataset(data_dir = thetaDir,
                             grid_dir = gridDir,
                             extra_variables = dict(SSH = dict(dims=['k','j','i'],
                                                               attrs = dict(standard_name="SST",
                                                                            long_name="Sea Surface Temperature",
                                                                            units="degC"))))
temp["SST"] = temp["SSH"]
temp = temp.drop(["SSH"])
    
temp_ext = xmitgcm.open_mdsdataset(data_dir = thetaDir_ext,
                                 grid_dir = gridDir,
                                 extra_variables = dict(SST = dict(dims=['k','j','i'],
                                                                   attrs = dict(standard_name="SST",
                                                                                long_name="Sea Surface Temperature",
                                                                                units="degC"))))

ds_r5 = xr.concat([temp, temp_ext], "time")

In [3]:
ds_SST = xr.open_dataset(mainDir_r5+'/SST_all.nc')
SST = ds_SST['SST'].data

In [4]:
hFacC = ecco.read_llc_to_tiles(gridDir, 'hFacC.data')
hFacC_mask = hFacC > 0
hFacC_mask = hFacC_mask.astype(float)

XC = ds_r4['XC'].data
YC = ds_r4['YC'].data

latMask = YC > -20.0
latMask = latMask.astype(float)

maskFinal = hFacC_mask * latMask
NaNmaskFinal = np.copy(maskFinal)
NaNmaskFinal[NaNmaskFinal == 0] = np.nan

da_XC = xr.DataArray(
    data=ds_r4['XC'].data,
    dims=["tile", "j", "i"],
    coords=dict(
        tile = ds_r4['tile'].data,
        j    = ds_r4['j'].data,
        i    = ds_r4['i'].data,
    ),
    attrs=dict(description="XC"),
)

da_YC = xr.DataArray(
    data=ds_r4['YC'].data,
    dims=["tile", "j", "i"],
    coords=dict(
        tile = ds_r4['tile'].data,
        j    = ds_r4['j'].data,
        i    = ds_r4['i'].data,
    ),
    attrs=dict(description="YC"),
)

da_hFacC_mask = xr.DataArray(
    data=hFacC_mask,
    dims=["tile", "j", "i"],
    coords=dict(
        tile = ds_r4['tile'].data,
        j    = ds_r4['j'].data,
        i    = ds_r4['i'].data,
    ),
    attrs=dict(description="hFacC mask 2D 1 if > 0, else 0"),
)

da_latMask = xr.DataArray(
    data=latMask,
    dims=["tile", "j", "i"],
    coords=dict(
        tile = ds_r4['tile'].data,
        j    = ds_r4['j'].data,
        i    = ds_r4['i'].data,
    ),
    attrs=dict(description="Latitude Mask 1 if > -20, else 0"),
)

da_maskFinal = xr.DataArray(
    data=maskFinal,
    dims=["tile", "j", "i"],
    coords=dict(
        tile = ds_r4['tile'].data,
        j    = ds_r4['j'].data,
        i    = ds_r4['i'].data,
    ),
    attrs=dict(description="Mask 2D 1 if > 0, else 0"),
)

da_NaNmaskFinal = xr.DataArray(
    data=NaNmaskFinal,
    dims=["tile", "j", "i"],
    coords=dict(
        tile = ds_r4['tile'].data,
        j    = ds_r4['j'].data,
        i    = ds_r4['i'].data,
    ),
    attrs=dict(description="Mask 2D True if > 0, else NaN"),
)

wetpoints = np.nonzero(maskFinal.data)
da_wetpoints = xr.DataArray(
    data=np.asarray(wetpoints),
    dims=["wetpoints_dim", "num_wetpoints"],
    coords=dict(
        wetpoints_dim = np.arange(np.asarray(wetpoints).shape[0], dtype = int),
        num_wetpoints = np.arange(np.asarray(wetpoints).shape[1], dtype = int),
    ),
    attrs=dict(description="indices of wetpoints in the order (tile, j, i) in the three rows"),
)

da_SST = xr.DataArray(
    data=SST,
    dims=["time", "tile", "j", "i"],
    coords=dict(
        time = ds_r5['time'].data[:SST.shape[0]],
        tile = ds_r4['tile'].data,
        j    = ds_r4['j'].data,
        i    = ds_r4['i'].data,
    ),
    attrs=dict(description="SST field in llc format"),
)

ds = xr.Dataset()
ds = ds.assign(XC           = da_XC,
               YC           = da_YC,
               hFacC_mask   = da_hFacC_mask,
               latMask      = da_latMask,
               maskFinal    = da_maskFinal,
               NaNmaskFinal = da_NaNmaskFinal,
               wetpoints    = da_wetpoints,
               SST          = da_SST)

load_binary_array: loading file /work/07665/shrey911/ls6/LRP_eccov4r5_data/GRID/hFacC.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4


In [5]:
def anomalize_new(field, num_years = 31, first_leap_year_idx = 0):
    
    leap_yr_offsets_jan_feb   = np.array([0,1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4,5,5,5,5,6,6,6,6,7,7,7,7,8,8])
    leap_yr_offsets_after_feb = np.array([1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4,5,5,5,5,6,6,6,6,7,7,7,7,8,8,8])

    if len(field.shape) > 1:
        seasonal_trend = np.zeros((366, field.shape[1]))
    else:
        seasonal_trend = np.zeros((366,))

    #### Calculate seasonal trend
    
    # Jan 1 - Feb 28
    for d in range(59):
        same_cal_days_idx=[d+365*year+leap_yr_offsets_jan_feb[year] for year in range(num_years)]
        # Remove mean
        field[same_cal_days_idx] = scipy.signal.detrend(field[same_cal_days_idx], 
                                                        axis=0, 
                                                        type='constant', 
                                                        overwrite_data=False)
        # Remove linear trend
        field[same_cal_days_idx] = scipy.signal.detrend(field[same_cal_days_idx], 
                                                        axis=0, 
                                                        type='linear', 
                                                        overwrite_data=False)
    
    # Feb 29 starting 1996, so year 2 in 0-indexing
    same_cal_days_idx=[365*year+59+int(year/4) for year in range(first_leap_year_idx,num_years,4)]
    # Remove mean
    field[same_cal_days_idx] = scipy.signal.detrend(field[same_cal_days_idx], 
                                                    axis=0, 
                                                    type='constant', 
                                                    overwrite_data=False)
    # Remove linear trend
    field[same_cal_days_idx] = scipy.signal.detrend(field[same_cal_days_idx], 
                                                    axis=0, 
                                                    type='linear', 
                                                    overwrite_data=False)
            
    # Mar 1 - Dec 31
    for d in range(60,366):
        same_cal_days_idx=[d-1+365*year+leap_yr_offsets_after_feb[year] for year in range(num_years)]
        # Remove mean
        field[same_cal_days_idx] = scipy.signal.detrend(field[same_cal_days_idx], 
                                                        axis=0, 
                                                        type='constant', 
                                                        overwrite_data=False)
        # Remove linear trend
        field[same_cal_days_idx] = scipy.signal.detrend(field[same_cal_days_idx], 
                                                        axis=0, 
                                                        type='linear', 
                                                        overwrite_data=False)

    return field

In [6]:
X = ds['SST'].data[:,wetpoints[0],wetpoints[1],wetpoints[2]].copy()
X = anomalize_new(X)
X_full = X.copy()
X = X[30:-30]

y = ds['SST'].isel(tile = 10, j = 1, i = 43).data.copy()
y = anomalize_new(y)
# https://stackoverflow.com/questions/13728392/moving-average-or-running-mean
y = np.convolve(y, np.ones(61)/61, mode='valid')
oneHotCost = np.zeros((y.shape[0], 2), dtype = int)
oneHotCost[:,0] = y >= 0.0
oneHotCost[:,1] = y <  0.0

In [7]:
da_X = xr.DataArray(
    data=X,
    dims=["time_allData", "num_wetpoints"],
    coords=dict(
        time_allData  = ds['time'].data[30:-30],
        num_wetpoints = ds['num_wetpoints'].data,
    ),
    attrs=dict(description="All data as matrix X; deseasoned, delinearized and mean removed."),
)

da_y = xr.DataArray(
    data=y,
    dims=["time_allData"],
    coords=dict(
        time_allData  = ds['time'].data[30:-30],
    ),
    attrs=dict(description="All cost function y; deseasoned, delinearized and mean removed."),
)

da_X_full = xr.DataArray(
    data=X_full,
    dims=["time", "num_wetpoints"],
    coords=dict(
        time          = ds['time'],
        num_wetpoints = ds['num_wetpoints'].data,
    ),
    attrs=dict(description="All data without accounting for conv filter as matrix X_full; deseasoned, delinearized and mean removed."),
)

da_oneHotCost = xr.DataArray(
    data=oneHotCost,
    dims=["time_allData", "NN_output_dim"],
    coords=dict(
        time_allData  = ds['time'].data[30:-30],
        NN_output_dim = np.array([0,1]),
    ),
    attrs=dict(description="All cost function as one-hot vector."),
)

ds = ds.assign(X          = da_X,
               y          = da_y,
               X_full     = da_X_full,
               oneHotCost = da_oneHotCost)

## Optimal Input

In [8]:
def quickSetup(X, y,
               test_split_frac,
               val_split_frac,
               lagSteps):
    
    result = {}

    idx = int(X.shape[0]*(1-test_split_frac))
    X_train = X[:idx]
    oneHotCost_train = oneHotCost[:idx]
    X_test = X[idx:]
    oneHotCost_test = oneHotCost[idx:]

    keras.backend.clear_session()

    # Split the data into train and validation sets
    if lagSteps > 0:
        x_t, x_v, oneHotCost_t, oneHotCost_v = train_test_split(X_train[:-lagSteps], oneHotCost_train[lagSteps:], 
                                                                test_size=val_split_frac, shuffle= True, random_state=42)
    elif lagSteps == 0:
        x_t, x_v, oneHotCost_t, oneHotCost_v = train_test_split(X_train, oneHotCost_train, 
                                                                test_size=val_split_frac, shuffle= True, random_state=42)
    else:
        x_t, x_v, oneHotCost_t, oneHotCost_v = train_test_split(X_train[-lagSteps:], oneHotCost_train[:lagSteps], 
                                                                test_size=val_split_frac, shuffle= True, random_state=42)

    best_model = keras.models.load_model(f'LRP_output_forHelen/saved_models/model{lagSteps}_noL1.h5', 
                                         custom_objects={'metricF1': metricF1})

    result['QoI_true_full'] = np.argmax(oneHotCost, axis=1)
                   
    if lagSteps > 0:
        QoI_predict = np.argmax(best_model.predict(X[:-lagSteps]), axis=1)
        QoI_true = np.argmax(oneHotCost[lagSteps:], axis=1)
    elif lagSteps == 0:
        QoI_predict = np.argmax(best_model.predict(X), axis=1)
        QoI_true = np.argmax(oneHotCost, axis=1)
    else:
        QoI_predict = np.argmax(best_model.predict(X[-lagSteps:]), axis=1)
        QoI_true = np.argmax(oneHotCost[:lagSteps], axis=1)

    result['QoI_true_trunc'] = QoI_true.copy()
    result['QoI_predict_trunc'] = QoI_predict.copy()
                   
    result['QoI_success_predict_flag'] = QoI_true == QoI_predict
    result['Accuracy'] = accuracy_score(QoI_true, QoI_predict)
    result['F1_Score'] = f1_score(QoI_true, QoI_predict, average='binary')

    if lagSteps > 0:
        QoI_predict = np.argmax(best_model.predict(X_train[:-lagSteps]), axis=1)
        QoI_true = np.argmax(oneHotCost_train[lagSteps:], axis=1)
    elif lagSteps == 0:
        QoI_predict = np.argmax(best_model.predict(X_train), axis=1)
        QoI_true = np.argmax(oneHotCost_train, axis=1)
    else:
        QoI_predict = np.argmax(best_model.predict(X_train[-lagSteps:]), axis=1)
        QoI_true = np.argmax(oneHotCost_train[:lagSteps], axis=1)

    result['QoI_true_train_trunc'] = QoI_true.copy()
    result['QoI_predict_train_trunc'] = QoI_predict.copy()
                   
    result['QoI_success_predict_flag_train'] = QoI_true == QoI_predict
    result['Accuracy_train'] = accuracy_score(QoI_true, QoI_predict)
    result['F1_Score_train'] = f1_score(QoI_true, QoI_predict, average='binary')

    if lagSteps > 0:
        QoI_predict = np.argmax(best_model.predict(X_test[:-lagSteps]), axis=1)
        QoI_true = np.argmax(oneHotCost_test[lagSteps:], axis=1)
    elif lagSteps == 0:
        QoI_predict = np.argmax(best_model.predict(X_test), axis=1)
        QoI_true = np.argmax(oneHotCost_test, axis=1)
    else:
        QoI_predict = np.argmax(best_model.predict(X_test[-lagSteps:]), axis=1)
        QoI_true = np.argmax(oneHotCost_test[:lagSteps], axis=1)

    result['QoI_true_test_trunc'] = QoI_true.copy()
    result['QoI_predict_test_trunc'] = QoI_predict.copy()
                   
    result['QoI_success_predict_flag_test'] = QoI_true == QoI_predict
    result['Accuracy_test'] = accuracy_score(QoI_true, QoI_predict)
    result['F1_Score_test'] = f1_score(QoI_true, QoI_predict, average='binary')

    print(f"Lag {lagSteps}, Accuracy {result['Accuracy']*100:2.2f}, F1 Score {result['F1_Score']:2.2f}, Accuracy Train {result['Accuracy_train']*100:2.2f}, F1 Score Train {result['F1_Score_train']:2.2f}, Accuracy Test {result['Accuracy_test']*100:2.2f}, F1 Score Test {result['F1_Score_test']:2.2f}")

    return result

In [9]:
lagStepsList = [-60,-30,0,30,60,90,120,150,180]
result_dict = {}
for i in range(len(lagStepsList)):

    print(f'Lag: {lagStepsList[i]} days, for Theta')
    # 365*6-30+1 = 2161
    result_dict[f'lag{lagStepsList[i]}'] = quickSetup(X, y, 2161.0/11263.0,
                                             0.2,
                                             lagStepsList[i])

Lag: -60 days, for Theta


2024-12-09 20:30:55.642210: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-12-09 20:30:55.644549: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-12-09 20:30:55.646629: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-12-09 20:30:55.648751: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-12-09 20:30:55.650848: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

Lag -60, Accuracy 91.87, F1 Score 0.93, Accuracy Train 99.47, F1 Score Train 1.00, Accuracy Test 61.78, F1 Score Test 0.69
Lag: -30 days, for Theta


2024-12-09 20:31:13.360757: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-12-09 20:31:13.363227: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-12-09 20:31:13.365350: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-12-09 20:31:13.367475: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-12-09 20:31:13.369551: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

Lag -30, Accuracy 91.34, F1 Score 0.92, Accuracy Train 99.50, F1 Score Train 1.00, Accuracy Test 56.45, F1 Score Test 0.53
Lag: 0 days, for Theta


...

Lag 120, Accuracy 92.25, F1 Score 0.93, Accuracy Train 99.39, F1 Score Train 0.99, Accuracy Test 62.22, F1 Score Test 0.64
Lag: 150 days, for Theta


2024-12-09 20:31:22.708042: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-12-09 20:31:22.710388: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-12-09 20:31:22.712464: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-12-09 20:31:22.714599: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-12-09 20:31:22.716672: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

Lag 150, Accuracy 94.52, F1 Score 0.95, Accuracy Train 99.46, F1 Score Train 1.00, Accuracy Test 72.10, F1 Score Test 0.74
Lag: 180 days, for Theta


2024-12-09 20:31:24.146511: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-12-09 20:31:24.148842: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-12-09 20:31:24.150918: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-12-09 20:31:24.153073: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-12-09 20:31:24.155148: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

Lag 180, Accuracy 93.77, F1 Score 0.94, Accuracy Train 99.38, F1 Score Train 0.99, Accuracy Test 67.90, F1 Score Test 0.72


In [10]:
ds = xr.Dataset()
ds_1 = xr.Dataset()

da_QoI_true_full_minus60 = xr.DataArray(result_dict[f'lag-60']['QoI_true_full'].copy(), dims=("minus60_full"))
da_QoI_true_trunc_minus60 = xr.DataArray(result_dict[f'lag-60']['QoI_true_trunc'].copy(), dims=("minus60_trunc"))
da_QoI_predict_trunc_minus60 = xr.DataArray(result_dict[f'lag-60']['QoI_predict_trunc'].copy(), dims=("minus60_trunc"))
da_QoI_true_train_trunc_minus60 = xr.DataArray(result_dict[f'lag-60']['QoI_true_train_trunc'].copy(), dims=("minus60_train_trunc"))
da_QoI_predict_train_trunc_minus60 = xr.DataArray(result_dict[f'lag-60']['QoI_predict_train_trunc'].copy(), dims=("minus60_train_trunc"))
da_QoI_true_test_trunc_minus60 = xr.DataArray(result_dict[f'lag-60']['QoI_true_test_trunc'].copy(), dims=("minus60_test_trunc"))
da_QoI_predict_test_trunc_minus60 = xr.DataArray(result_dict[f'lag-60']['QoI_predict_test_trunc'].copy(), dims=("minus60_test_trunc"))
da_accuracy_minus60 = xr.DataArray(result_dict[f'lag-60']['Accuracy'].copy())
da_f1score_minus60 = xr.DataArray(result_dict[f'lag-60']['F1_Score'].copy())
da_accuracy_train_minus60 = xr.DataArray(result_dict[f'lag-60']['Accuracy_train'].copy())
da_f1score_train_minus60 = xr.DataArray(result_dict[f'lag-60']['F1_Score_train'].copy())
da_accuracy_test_minus60 = xr.DataArray(result_dict[f'lag-60']['Accuracy_test'].copy())
da_f1score_test_minus60 = xr.DataArray(result_dict[f'lag-60']['F1_Score_test'].copy())

ds = ds.assign(QoI_true_full_minus60 = da_QoI_true_full_minus60,
               QoI_true_trunc_minus60 = da_QoI_true_trunc_minus60,
               QoI_predict_trunc_minus60 = da_QoI_predict_trunc_minus60,
               QoI_true_train_trunc_minus60 = da_QoI_true_train_trunc_minus60,
               QoI_predict_train_trunc_minus60 = da_QoI_predict_train_trunc_minus60,
               QoI_true_test_trunc_minus60 = da_QoI_true_test_trunc_minus60,
               QoI_predict_test_trunc_minus60 = da_QoI_predict_test_trunc_minus60)
ds_1 = ds_1.assign(accuracy_minus60 = da_accuracy_minus60,
                   f1score_minus60 = da_f1score_minus60,
                   accuracy_train_minus60 = da_accuracy_train_minus60,
                   f1score_train_minus60 = da_f1score_train_minus60,
                   accuracy_test_minus60 = da_accuracy_test_minus60,
                   f1score_test_minus60 = da_f1score_test_minus60)

da_QoI_true_full_minus30 = xr.DataArray(result_dict[f'lag-30']['QoI_true_full'].copy(), dims=("minus30_full"))
da_QoI_true_trunc_minus30 = xr.DataArray(result_dict[f'lag-30']['QoI_true_trunc'].copy(), dims=("minus30_trunc"))
da_QoI_predict_trunc_minus30 = xr.DataArray(result_dict[f'lag-30']['QoI_predict_trunc'].copy(), dims=("minus30_trunc"))
da_QoI_true_train_trunc_minus30 = xr.DataArray(result_dict[f'lag-30']['QoI_true_train_trunc'].copy(), dims=("minus30_train_trunc"))
da_QoI_predict_train_trunc_minus30 = xr.DataArray(result_dict[f'lag-30']['QoI_predict_train_trunc'].copy(), dims=("minus30_train_trunc"))
da_QoI_true_test_trunc_minus30 = xr.DataArray(result_dict[f'lag-30']['QoI_true_test_trunc'].copy(), dims=("minus30_test_trunc"))
da_QoI_predict_test_trunc_minus30 = xr.DataArray(result_dict[f'lag-30']['QoI_predict_test_trunc'].copy(), dims=("minus30_test_trunc"))
da_accuracy_minus30 = xr.DataArray(result_dict[f'lag-30']['Accuracy'].copy())
da_f1score_minus30 = xr.DataArray(result_dict[f'lag-30']['F1_Score'].copy())
da_accuracy_train_minus30 = xr.DataArray(result_dict[f'lag-30']['Accuracy_train'].copy())
da_f1score_train_minus30 = xr.DataArray(result_dict[f'lag-30']['F1_Score_train'].copy())
da_accuracy_test_minus30 = xr.DataArray(result_dict[f'lag-30']['Accuracy_test'].copy())
da_f1score_test_minus30 = xr.DataArray(result_dict[f'lag-30']['F1_Score_test'].copy())

ds = ds.assign(QoI_true_full_minus30 = da_QoI_true_full_minus30,
               QoI_true_trunc_minus30 = da_QoI_true_trunc_minus30,
               QoI_predict_trunc_minus30 = da_QoI_predict_trunc_minus30,
               QoI_true_train_trunc_minus30 = da_QoI_true_train_trunc_minus30,
               QoI_predict_train_trunc_minus30 = da_QoI_predict_train_trunc_minus30,
               QoI_true_test_trunc_minus30 = da_QoI_true_test_trunc_minus30,
               QoI_predict_test_trunc_minus30 = da_QoI_predict_test_trunc_minus30)
ds_1 = ds_1.assign(accuracy_minus30 = da_accuracy_minus30,
                   f1score_minus30 = da_f1score_minus30,
                   accuracy_train_minus30 = da_accuracy_train_minus30,
                   f1score_train_minus30 = da_f1score_train_minus30,
                   accuracy_test_minus30 = da_accuracy_test_minus30,
                   f1score_test_minus30 = da_f1score_test_minus30)

da_QoI_true_full_0 = xr.DataArray(result_dict[f'lag0']['QoI_true_full'].copy(), dims=("0_full"))
da_QoI_true_trunc_0 = xr.DataArray(result_dict[f'lag0']['QoI_true_trunc'].copy(), dims=("0_trunc"))
da_QoI_predict_trunc_0 = xr.DataArray(result_dict[f'lag0']['QoI_predict_trunc'].copy(), dims=("0_trunc"))
da_QoI_true_train_trunc_0 = xr.DataArray(result_dict[f'lag0']['QoI_true_train_trunc'].copy(), dims=("0_train_trunc"))
da_QoI_predict_train_trunc_0 = xr.DataArray(result_dict[f'lag0']['QoI_predict_train_trunc'].copy(), dims=("0_train_trunc"))
da_QoI_true_test_trunc_0 = xr.DataArray(result_dict[f'lag0']['QoI_true_test_trunc'].copy(), dims=("0_test_trunc"))
da_QoI_predict_test_trunc_0 = xr.DataArray(result_dict[f'lag0']['QoI_predict_test_trunc'].copy(), dims=("0_test_trunc"))
da_accuracy_0 = xr.DataArray(result_dict[f'lag0']['Accuracy'].copy())
da_f1score_0 = xr.DataArray(result_dict[f'lag0']['F1_Score'].copy())
da_accuracy_train_0 = xr.DataArray(result_dict[f'lag0']['Accuracy_train'].copy())
da_f1score_train_0 = xr.DataArray(result_dict[f'lag0']['F1_Score_train'].copy())
da_accuracy_test_0 = xr.DataArray(result_dict[f'lag0']['Accuracy_test'].copy())
da_f1score_test_0 = xr.DataArray(result_dict[f'lag0']['F1_Score_test'].copy())

ds = ds.assign(QoI_true_full_0 = da_QoI_true_full_0,
               QoI_true_trunc_0 = da_QoI_true_trunc_0,
               QoI_predict_trunc_0 = da_QoI_predict_trunc_0,
               QoI_true_train_trunc_0 = da_QoI_true_train_trunc_0,
               QoI_predict_train_trunc_0 = da_QoI_predict_train_trunc_0,
               QoI_true_test_trunc_0 = da_QoI_true_test_trunc_0,
               QoI_predict_test_trunc_0 = da_QoI_predict_test_trunc_0)
ds_1 = ds_1.assign(accuracy_0 = da_accuracy_0,
                   f1score_0 = da_f1score_0,
                   accuracy_train_0 = da_accuracy_train_0,
                   f1score_train_0 = da_f1score_train_0,
                   accuracy_test_0 = da_accuracy_test_0,
                   f1score_test_0 = da_f1score_test_0)

da_QoI_true_full_30 = xr.DataArray(result_dict[f'lag30']['QoI_true_full'].copy(), dims=("30_full"))
da_QoI_true_trunc_30 = xr.DataArray(result_dict[f'lag30']['QoI_true_trunc'].copy(), dims=("30_trunc"))
da_QoI_predict_trunc_30 = xr.DataArray(result_dict[f'lag30']['QoI_predict_trunc'].copy(), dims=("30_trunc"))
da_QoI_true_train_trunc_30 = xr.DataArray(result_dict[f'lag30']['QoI_true_train_trunc'].copy(), dims=("30_train_trunc"))
da_QoI_predict_train_trunc_30 = xr.DataArray(result_dict[f'lag30']['QoI_predict_train_trunc'].copy(), dims=("30_train_trunc"))
da_QoI_true_test_trunc_30 = xr.DataArray(result_dict[f'lag30']['QoI_true_test_trunc'].copy(), dims=("30_test_trunc"))
da_QoI_predict_test_trunc_30 = xr.DataArray(result_dict[f'lag30']['QoI_predict_test_trunc'].copy(), dims=("30_test_trunc"))
da_accuracy_30 = xr.DataArray(result_dict[f'lag30']['Accuracy'].copy())
da_f1score_30 = xr.DataArray(result_dict[f'lag30']['F1_Score'].copy())
da_accuracy_train_30 = xr.DataArray(result_dict[f'lag30']['Accuracy_train'].copy())
da_f1score_train_30 = xr.DataArray(result_dict[f'lag30']['F1_Score_train'].copy())
da_accuracy_test_30 = xr.DataArray(result_dict[f'lag30']['Accuracy_test'].copy())
da_f1score_test_30 = xr.DataArray(result_dict[f'lag30']['F1_Score_test'].copy())

ds = ds.assign(QoI_true_full_30 = da_QoI_true_full_30,
               QoI_true_trunc_30 = da_QoI_true_trunc_30,
               QoI_predict_trunc_30 = da_QoI_predict_trunc_30,
               QoI_true_train_trunc_30 = da_QoI_true_train_trunc_30,
               QoI_predict_train_trunc_30 = da_QoI_predict_train_trunc_30,
               QoI_true_test_trunc_30 = da_QoI_true_test_trunc_30,
               QoI_predict_test_trunc_30 = da_QoI_predict_test_trunc_30)
ds_1 = ds_1.assign(accuracy_30 = da_accuracy_30,
                   f1score_30 = da_f1score_30,
                   accuracy_train_30 = da_accuracy_train_30,
                   f1score_train_30 = da_f1score_train_30,
                   accuracy_test_30 = da_accuracy_test_30,
                   f1score_test_30 = da_f1score_test_30)

da_QoI_true_full_60 = xr.DataArray(result_dict[f'lag60']['QoI_true_full'].copy(), dims=("60_full"))
da_QoI_true_trunc_60 = xr.DataArray(result_dict[f'lag60']['QoI_true_trunc'].copy(), dims=("60_trunc"))
da_QoI_predict_trunc_60 = xr.DataArray(result_dict[f'lag60']['QoI_predict_trunc'].copy(), dims=("60_trunc"))
da_QoI_true_train_trunc_60 = xr.DataArray(result_dict[f'lag60']['QoI_true_train_trunc'].copy(), dims=("60_train_trunc"))
da_QoI_predict_train_trunc_60 = xr.DataArray(result_dict[f'lag60']['QoI_predict_train_trunc'].copy(), dims=("60_train_trunc"))
da_QoI_true_test_trunc_60 = xr.DataArray(result_dict[f'lag60']['QoI_true_test_trunc'].copy(), dims=("60_test_trunc"))
da_QoI_predict_test_trunc_60 = xr.DataArray(result_dict[f'lag60']['QoI_predict_test_trunc'].copy(), dims=("60_test_trunc"))
da_accuracy_60 = xr.DataArray(result_dict[f'lag60']['Accuracy'].copy())
da_f1score_60 = xr.DataArray(result_dict[f'lag60']['F1_Score'].copy())
da_accuracy_train_60 = xr.DataArray(result_dict[f'lag60']['Accuracy_train'].copy())
da_f1score_train_60 = xr.DataArray(result_dict[f'lag60']['F1_Score_train'].copy())
da_accuracy_test_60 = xr.DataArray(result_dict[f'lag60']['Accuracy_test'].copy())
da_f1score_test_60 = xr.DataArray(result_dict[f'lag60']['F1_Score_test'].copy())

ds = ds.assign(QoI_true_full_60 = da_QoI_true_full_60,
               QoI_true_trunc_60 = da_QoI_true_trunc_60,
               QoI_predict_trunc_60 = da_QoI_predict_trunc_60,
               QoI_true_train_trunc_60 = da_QoI_true_train_trunc_60,
               QoI_predict_train_trunc_60 = da_QoI_predict_train_trunc_60,
               QoI_true_test_trunc_60 = da_QoI_true_test_trunc_60,
               QoI_predict_test_trunc_60 = da_QoI_predict_test_trunc_60)
ds_1 = ds_1.assign(accuracy_60 = da_accuracy_60,
                   f1score_60 = da_f1score_60,
                   accuracy_train_60 = da_accuracy_train_60,
                   f1score_train_60 = da_f1score_train_60,
                   accuracy_test_60 = da_accuracy_test_60,
                   f1score_test_60 = da_f1score_test_60)

da_QoI_true_full_90 = xr.DataArray(result_dict[f'lag90']['QoI_true_full'].copy(), dims=("90_full"))
da_QoI_true_trunc_90 = xr.DataArray(result_dict[f'lag90']['QoI_true_trunc'].copy(), dims=("90_trunc"))
da_QoI_predict_trunc_90 = xr.DataArray(result_dict[f'lag90']['QoI_predict_trunc'].copy(), dims=("90_trunc"))
da_QoI_true_train_trunc_90 = xr.DataArray(result_dict[f'lag90']['QoI_true_train_trunc'].copy(), dims=("90_train_trunc"))
da_QoI_predict_train_trunc_90 = xr.DataArray(result_dict[f'lag90']['QoI_predict_train_trunc'].copy(), dims=("90_train_trunc"))
da_QoI_true_test_trunc_90 = xr.DataArray(result_dict[f'lag90']['QoI_true_test_trunc'].copy(), dims=("90_test_trunc"))
da_QoI_predict_test_trunc_90 = xr.DataArray(result_dict[f'lag90']['QoI_predict_test_trunc'].copy(), dims=("90_test_trunc"))
da_accuracy_90 = xr.DataArray(result_dict[f'lag90']['Accuracy'].copy())
da_f1score_90 = xr.DataArray(result_dict[f'lag90']['F1_Score'].copy())
da_accuracy_train_90 = xr.DataArray(result_dict[f'lag90']['Accuracy_train'].copy())
da_f1score_train_90 = xr.DataArray(result_dict[f'lag90']['F1_Score_train'].copy())
da_accuracy_test_90 = xr.DataArray(result_dict[f'lag90']['Accuracy_test'].copy())
da_f1score_test_90 = xr.DataArray(result_dict[f'lag90']['F1_Score_test'].copy())

ds = ds.assign(QoI_true_full_90 = da_QoI_true_full_90,
               QoI_true_trunc_90 = da_QoI_true_trunc_90,
               QoI_predict_trunc_90 = da_QoI_predict_trunc_90,
               QoI_true_train_trunc_90 = da_QoI_true_train_trunc_90,
               QoI_predict_train_trunc_90 = da_QoI_predict_train_trunc_90,
               QoI_true_test_trunc_90 = da_QoI_true_test_trunc_90,
               QoI_predict_test_trunc_90 = da_QoI_predict_test_trunc_90)
ds_1 = ds_1.assign(accuracy_90 = da_accuracy_90,
                   f1score_90 = da_f1score_90,
                   accuracy_train_90 = da_accuracy_train_90,
                   f1score_train_90 = da_f1score_train_90,
                   accuracy_test_90 = da_accuracy_test_90,
                   f1score_test_90 = da_f1score_test_90)

da_QoI_true_full_120 = xr.DataArray(result_dict[f'lag120']['QoI_true_full'].copy(), dims=("120_full"))
da_QoI_true_trunc_120 = xr.DataArray(result_dict[f'lag120']['QoI_true_trunc'].copy(), dims=("120_trunc"))
da_QoI_predict_trunc_120 = xr.DataArray(result_dict[f'lag120']['QoI_predict_trunc'].copy(), dims=("120_trunc"))
da_QoI_true_train_trunc_120 = xr.DataArray(result_dict[f'lag120']['QoI_true_train_trunc'].copy(), dims=("120_train_trunc"))
da_QoI_predict_train_trunc_120 = xr.DataArray(result_dict[f'lag120']['QoI_predict_train_trunc'].copy(), dims=("120_train_trunc"))
da_QoI_true_test_trunc_120 = xr.DataArray(result_dict[f'lag120']['QoI_true_test_trunc'].copy(), dims=("120_test_trunc"))
da_QoI_predict_test_trunc_120 = xr.DataArray(result_dict[f'lag120']['QoI_predict_test_trunc'].copy(), dims=("120_test_trunc"))
da_accuracy_120 = xr.DataArray(result_dict[f'lag120']['Accuracy'].copy())
da_f1score_120 = xr.DataArray(result_dict[f'lag120']['F1_Score'].copy())
da_accuracy_train_120 = xr.DataArray(result_dict[f'lag120']['Accuracy_train'].copy())
da_f1score_train_120 = xr.DataArray(result_dict[f'lag120']['F1_Score_train'].copy())
da_accuracy_test_120 = xr.DataArray(result_dict[f'lag120']['Accuracy_test'].copy())
da_f1score_test_120 = xr.DataArray(result_dict[f'lag120']['F1_Score_test'].copy())

ds = ds.assign(QoI_true_full_120 = da_QoI_true_full_120,
               QoI_true_trunc_120 = da_QoI_true_trunc_120,
               QoI_predict_trunc_120 = da_QoI_predict_trunc_120,
               QoI_true_train_trunc_120 = da_QoI_true_train_trunc_120,
               QoI_predict_train_trunc_120 = da_QoI_predict_train_trunc_120,
               QoI_true_test_trunc_120 = da_QoI_true_test_trunc_120,
               QoI_predict_test_trunc_120 = da_QoI_predict_test_trunc_120)
ds_1 = ds_1.assign(accuracy_120 = da_accuracy_120,
                   f1score_120 = da_f1score_120,
                   accuracy_train_120 = da_accuracy_train_120,
                   f1score_train_120 = da_f1score_train_120,
                   accuracy_test_120 = da_accuracy_test_120,
                   f1score_test_120 = da_f1score_test_120)

da_QoI_true_full_150 = xr.DataArray(result_dict[f'lag150']['QoI_true_full'].copy(), dims=("150_full"))
da_QoI_true_trunc_150 = xr.DataArray(result_dict[f'lag150']['QoI_true_trunc'].copy(), dims=("150_trunc"))
da_QoI_predict_trunc_150 = xr.DataArray(result_dict[f'lag150']['QoI_predict_trunc'].copy(), dims=("150_trunc"))
da_QoI_true_train_trunc_150 = xr.DataArray(result_dict[f'lag150']['QoI_true_train_trunc'].copy(), dims=("150_train_trunc"))
da_QoI_predict_train_trunc_150 = xr.DataArray(result_dict[f'lag150']['QoI_predict_train_trunc'].copy(), dims=("150_train_trunc"))
da_QoI_true_test_trunc_150 = xr.DataArray(result_dict[f'lag150']['QoI_true_test_trunc'].copy(), dims=("150_test_trunc"))
da_QoI_predict_test_trunc_150 = xr.DataArray(result_dict[f'lag150']['QoI_predict_test_trunc'].copy(), dims=("150_test_trunc"))
da_accuracy_150 = xr.DataArray(result_dict[f'lag150']['Accuracy'].copy())
da_f1score_150 = xr.DataArray(result_dict[f'lag150']['F1_Score'].copy())
da_accuracy_train_150 = xr.DataArray(result_dict[f'lag150']['Accuracy_train'].copy())
da_f1score_train_150 = xr.DataArray(result_dict[f'lag150']['F1_Score_train'].copy())
da_accuracy_test_150 = xr.DataArray(result_dict[f'lag150']['Accuracy_test'].copy())
da_f1score_test_150 = xr.DataArray(result_dict[f'lag150']['F1_Score_test'].copy())

ds = ds.assign(QoI_true_full_150 = da_QoI_true_full_150,
               QoI_true_trunc_150 = da_QoI_true_trunc_150,
               QoI_predict_trunc_150 = da_QoI_predict_trunc_150,
               QoI_true_train_trunc_150 = da_QoI_true_train_trunc_150,
               QoI_predict_train_trunc_150 = da_QoI_predict_train_trunc_150,
               QoI_true_test_trunc_150 = da_QoI_true_test_trunc_150,
               QoI_predict_test_trunc_150 = da_QoI_predict_test_trunc_150)
ds_1 = ds_1.assign(accuracy_150 = da_accuracy_150,
                   f1score_150 = da_f1score_150,
                   accuracy_train_150 = da_accuracy_train_150,
                   f1score_train_150 = da_f1score_train_150,
                   accuracy_test_150 = da_accuracy_test_150,
                   f1score_test_150 = da_f1score_test_150)

da_QoI_true_full_180 = xr.DataArray(result_dict[f'lag180']['QoI_true_full'].copy(), dims=("180_full"))
da_QoI_true_trunc_180 = xr.DataArray(result_dict[f'lag180']['QoI_true_trunc'].copy(), dims=("180_trunc"))
da_QoI_predict_trunc_180 = xr.DataArray(result_dict[f'lag180']['QoI_predict_trunc'].copy(), dims=("180_trunc"))
da_QoI_true_train_trunc_180 = xr.DataArray(result_dict[f'lag180']['QoI_true_train_trunc'].copy(), dims=("180_train_trunc"))
da_QoI_predict_train_trunc_180 = xr.DataArray(result_dict[f'lag180']['QoI_predict_train_trunc'].copy(), dims=("180_train_trunc"))
da_QoI_true_test_trunc_180 = xr.DataArray(result_dict[f'lag180']['QoI_true_test_trunc'].copy(), dims=("180_test_trunc"))
da_QoI_predict_test_trunc_180 = xr.DataArray(result_dict[f'lag180']['QoI_predict_test_trunc'].copy(), dims=("180_test_trunc"))
da_accuracy_180 = xr.DataArray(result_dict[f'lag180']['Accuracy'].copy())
da_f1score_180 = xr.DataArray(result_dict[f'lag180']['F1_Score'].copy())
da_accuracy_train_180 = xr.DataArray(result_dict[f'lag180']['Accuracy_train'].copy())
da_f1score_train_180 = xr.DataArray(result_dict[f'lag180']['F1_Score_train'].copy())
da_accuracy_test_180 = xr.DataArray(result_dict[f'lag180']['Accuracy_test'].copy())
da_f1score_test_180 = xr.DataArray(result_dict[f'lag180']['F1_Score_test'].copy())

ds = ds.assign(QoI_true_full_180 = da_QoI_true_full_180,
               QoI_true_trunc_180 = da_QoI_true_trunc_180,
               QoI_predict_trunc_180 = da_QoI_predict_trunc_180,
               QoI_true_train_trunc_180 = da_QoI_true_train_trunc_180,
               QoI_predict_train_trunc_180 = da_QoI_predict_train_trunc_180,
               QoI_true_test_trunc_180 = da_QoI_true_test_trunc_180,
               QoI_predict_test_trunc_180 = da_QoI_predict_test_trunc_180)
ds_1 = ds_1.assign(accuracy_180 = da_accuracy_180,
                   f1score_180 = da_f1score_180,
                   accuracy_train_180 = da_accuracy_train_180,
                   f1score_train_180 = da_f1score_train_180,
                   accuracy_test_180 = da_accuracy_test_180,
                   f1score_test_180 = da_f1score_test_180)

In [11]:
ds.to_netcdf('LRP_output_forHelen/qoi_pred.nc')
ds_1.to_netcdf('LRP_output_forHelen/accuracy_f1.nc')